In [0]:
'''!pip install Cython
!pip install contextlib2
!pip install pillow
!pip install lxml
!pip install jupyter
!pip install matplotlib'''
!pip install wget

In [0]:
!apt-get -y install protobuf-compiler python-pil python-lxml

In [0]:
from google.colab import files
files.upload()

In [3]:
!mkdir images
%cd images
!mkdir train
!mkdir test
!ls

/content/images
test  train


In [4]:
%cd ..

/content


In [0]:
!python dataturks_to_PascalVOC.py Indian_Number_plates.json './images/train' './images/train'

INFO:root:Completed: 300 items done, 16 items ignored due to errors or for being skipped items.


In [0]:
import os
print(len(os.listdir('./images/train')))
#300 images and 300 voc files

600


In [0]:
import os
import PIL,cv2
from PIL import Image
lisdir = sorted(os.listdir('./images/train'))

for lis in lisdir:
  if lis[-4:] == 'jpeg':
    img = Image.open(os.path.join('./images/train',lis))
    os.remove(os.path.join('./images/train',lis))
    img = img.resize((600, 600), PIL.Image.ANTIALIAS)
    img.save(os.path.join('./images/train',lis))

In [0]:
print(len(os.listdir('./images/train')))

600


In [0]:
import os, shutil
path = "./images/train/"
moveto = "./images/test/"
files = os.listdir(path)
files.sort()
for i,f in enumerate(files):
    if i>499:
      src = path+f
      dst = moveto
      shutil.move(src,dst)

In [0]:
print(len(os.listdir('./images/train')))

500


In [0]:
import cv2
img = cv2.imread('./images/train/03273806-bb1e-48da-8c8b-a0133a90197a___2014-Skoda-Yeti-Test-Drive.jpg.jpeg')
from matplotlib import pyplot as plt
type(cv2.rectangle(img,(181,279),(379,351),(255,0,0),2))

numpy.ndarray

In [0]:
#XML to CSV
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET


def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
       
        tree = ET.parse(xml_file)
        root = tree.getroot()

        filename = root.find('filename').text
        width = root[4].find('width').text
        height = root[4].find('height').text
        class_ = 'plate'
        xmin = root[6].find('bndbox').find('xmin').text
        ymin = root[6].find('bndbox').find('ymin').text
        xmax = root[6].find('bndbox').find('xmax').text
        ymax = root[6].find('bndbox').find('ymax').text

        xml_list.append([filename,width,height,class_,xmin,ymin,xmax,ymax])
        
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df



image_path = './images/train'
xml_df = xml_to_csv(image_path)
xml_df.to_csv('./images/train_labels.csv', index=None)
#files.download('./images/train_labels.csv')

image_path = './images/test'
xml_df = xml_to_csv(image_path)
xml_df.to_csv('./images/test_labels.csv', index=None)
#files.download('./images/test_labels.csv')

In [0]:
!git clone https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 25469, done.
remote: Total 25469 (delta 0), reused 0 (delta 0), pack-reused 25469
Receiving objects: 100% (25469/25469), 507.96 MiB | 36.50 MiB/s, done.
Resolving deltas: 100% (15248/15248), done.
Checking out files: 100% (2883/2883), done.


In [0]:
#upload generate_tfrecord and move it
from google.colab import files
files.upload()

Saving generate_tfrecord.py to generate_tfrecord.py


{'generate_tfrecord.py': b"#generate tf records\r\n\r\nfrom __future__ import division\r\nfrom __future__ import print_function\r\nfrom __future__ import absolute_import\r\n\r\nimport os\r\nimport io\r\nimport pandas as pd\r\nimport tensorflow as tf\r\n\r\nfrom PIL import Image\r\nfrom object_detection.utils import dataset_util\r\nfrom collections import namedtuple, OrderedDict\r\n\r\nflags = tf.app.flags\r\nflags.DEFINE_string('csv_input', '', 'Path to the CSV input')\r\nflags.DEFINE_string('output_path', '', 'Path to output TFRecord')\r\nflags.DEFINE_string('image_dir', '', 'Path to images')\r\nFLAGS = flags.FLAGS\r\n\r\n\r\n# TO-DO replace this with label map\r\ndef class_text_to_int(row_label):\r\n    if row_label == 'plate':\r\n        return 1\r\n    else:\r\n        None\r\n\r\n\r\ndef split(df, group):\r\n    data = namedtuple('data', ['filename', 'object'])\r\n    gb = df.groupby(group)\r\n    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.

In [0]:
!mv generate_tfrecord.py ./models/research
%cd ./models/research/

/content/models/research


In [0]:
!python generate_tfrecord.py --csv_input=/content/images/train_labels.csv --output_path=train.record --image_dir=/content/images/train

Successfully created the TFRecords: /content/models/research/train.record


In [0]:
!python generate_tfrecord.py --csv_input=/content/images/test_labels.csv --output_path=test.record --image_dir=/content/images/test

Successfully created the TFRecords: /content/models/research/test.record


In [0]:
%cd /content/models/research/object_detection
!mkdir training

/content/models/research/object_detection


In [0]:
#files.download('./samples/configs/faster_rcnn_inception_v2_pets.config')
#edit the number of classes and the upload

In [0]:
%cd training

/content/models/research/object_detection/training


In [0]:
import wget
url = 'http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_v2_coco_2018_01_28.tar.gz'
filename = wget.download(url)

In [0]:
import tarfile
tar = tarfile.open('faster_rcnn_inception_v2_coco_2018_01_28.tar.gz', "r:gz")
tar.extractall()
tar.close()

In [0]:
#Upload the config file
files.upload()

Saving faster_rcnn_inception_v2_pets.config to faster_rcnn_inception_v2_pets.config


{'faster_rcnn_inception_v2_pets.config': b'# Faster R-CNN with Inception v2, configured for Oxford-IIIT Pets Dataset.\n# Users should configure the fine_tune_checkpoint field in the train config as\n# well as the label_map_path and input_path fields in the train_input_reader and\n# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that\n# should be configured.\n\nmodel {\n  faster_rcnn {\n    num_classes: 1\n    image_resizer {\n      keep_aspect_ratio_resizer {\n        min_dimension: 600\n        max_dimension: 1024\n      }\n    }\n    feature_extractor {\n      type: \'faster_rcnn_inception_v2\'\n      first_stage_features_stride: 16\n    }\n    first_stage_anchor_generator {\n      grid_anchor_generator {\n        scales: [0.25, 0.5, 1.0, 2.0]\n        aspect_ratios: [0.5, 1.0, 2.0]\n        height_stride: 16\n        width_stride: 16\n      }\n    }\n    first_stage_box_predictor_conv_hyperparams {\n      op: CONV\n      regularizer {\n        l2_regularize

In [0]:
%cd /content/models/research/object_detection/legacy

/content/models/research/object_detection/legacy


In [0]:
!cp train.py /content/models/research/
%cd /content/models/research/object_detection/protos

/content/models/research/object_detection/protos


In [0]:
#upload protoc files
files.upload()

Saving model_pb2.py to model_pb2.py
Saving multiscale_anchor_generator_pb2.py to multiscale_anchor_generator_pb2.py
Saving optimizer_pb2.py to optimizer_pb2.py
Saving pipeline_pb2.py to pipeline_pb2.py
Saving post_processing_pb2.py to post_processing_pb2.py
Saving preprocessor_pb2.py to preprocessor_pb2.py
Saving region_similarity_calculator_pb2.py to region_similarity_calculator_pb2.py
Saving square_box_coder_pb2.py to square_box_coder_pb2.py
Saving ssd_anchor_generator_pb2.py to ssd_anchor_generator_pb2.py
Saving ssd_pb2.py to ssd_pb2.py
Saving string_int_label_map_pb2.py to string_int_label_map_pb2.py
Saving train_pb2.py to train_pb2.py


{'model_pb2.py': b'# -*- coding: utf-8 -*-\n# Generated by the protocol buffer compiler.  DO NOT EDIT!\n# source: object_detection/protos/model.proto\n\nimport sys\n_b=sys.version_info[0]<3 and (lambda x:x) or (lambda x:x.encode(\'latin1\'))\nfrom google.protobuf import descriptor as _descriptor\nfrom google.protobuf import message as _message\nfrom google.protobuf import reflection as _reflection\nfrom google.protobuf import symbol_database as _symbol_database\n# @@protoc_insertion_point(imports)\n\n_sym_db = _symbol_database.Default()\n\n\nfrom object_detection.protos import faster_rcnn_pb2 as object__detection_dot_protos_dot_faster__rcnn__pb2\nfrom object_detection.protos import ssd_pb2 as object__detection_dot_protos_dot_ssd__pb2\n\n\nDESCRIPTOR = _descriptor.FileDescriptor(\n  name=\'object_detection/protos/model.proto\',\n  package=\'object_detection.protos\',\n  syntax=\'proto2\',\n  serialized_options=None,\n  serialized_pb=_b(\'\\n#object_detection/protos/model.proto\\x12\\x17

In [0]:
%cd /content/models/research

/content/models/research


In [0]:
import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

In [0]:
!python /content/models/research/object_detection/builders/model_builder_test.py


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

............s...
----------------------------------------------------------------------
Ran 16 tests in 0.068s

OK (skipped=1)


In [0]:
!python train.py --logtostderr --train_dir=/content/models/research/object_detection/training --pipeline_config_path=/content/models/research/object_detection/training/faster_rcnn_inception_v2_pets.config


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use object_detection/model_main.py.
Instructions for updating:
Please switch to tf.train.create_global_step
Instructions for updating:
Colocations handled automatically by placer.
Traceback (most recent call last):
  File "train.py", line 184, in <module>
    tf.app.run()
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/platform/app.py", line 125, in run
    _sys.exit(main(argv))
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/util/deprecation.py", line 324, in new_func
    return func(*args, **kwargs)
  File "train.py", line 180, in main
    graph_hook_fn=graph_rewriter_fn)
  File "/content/models/research/object_detection/legacy/trainer.py", line 280, in train
    train_config.prefetch_queue_c

In [0]:
% cd /content

In [0]:
files.upload()

In [0]:
!unzip protoc-3.2.0-linux-x86_64.zip -d protoc3
!sudo mv protoc3/bin/* /usr/local/bin/
!sudo mv protoc3/include/* /usr/local/include/
!sudo chwon [user] /usr/local/bin/protoc
!sudo chwon -R [user] /usr/local/include/google

In [0]:
!ls /bin/